## Import requirements

In [0]:
import tensorflow as tf
import numpy as np
import pandas as pd

## Fetch data

In [0]:
from google.colab import drive

In [3]:
drive.mount("/google_drive")

Drive already mounted at /google_drive; to attempt to forcibly remount, call drive.mount("/google_drive", force_remount=True).


In [4]:
!ln -s /google_drive/My\ Drive/datasets ~/user_datasets
!ls ~/user_datasets

ln: failed to create symbolic link '/root/user_datasets/datasets': Function not implemented
digit-recognizer


In [0]:
train_data_file = "~/user_datasets/digit-recognizer/train.csv"
test_data_file = "~/user_datasets/digit-recognizer/test.csv"

In [0]:
csv_train = pd.read_csv(train_data_file, sep=",")

In [7]:
csv_test = pd.read_csv(test_data_file, sep=",")
data_test = csv_test.iloc[:,:].values
csv_test.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,pixel39,...,pixel744,pixel745,pixel746,pixel747,pixel748,pixel749,pixel750,pixel751,pixel752,pixel753,pixel754,pixel755,pixel756,pixel757,pixel758,pixel759,pixel760,pixel761,pixel762,pixel763,pixel764,pixel765,pixel766,pixel767,pixel768,pixel769,pixel770,pixel771,pixel772,pixel773,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [8]:
print(data_test.shape)

(28000, 784)


## Pre-process data

### One-hot encoding

In [0]:
def dense2one_hot(label_dense, num_class):
    num_label = label_dense.shape[0]
    index_offset = np.arange(num_label) * num_class
    label_one_hot = np.zeros((num_label, num_class))
    label_one_hot.flat[index_offset + label_dense.ravel()] = 1
    return label_one_hot

In [0]:
def parse_train_csv(csv_source):
    buffer_csv = csv_source.sample(frac=1).reset_index(drop=True)
    data_train = buffer_csv.iloc[:, 1:].values.astype(np.float32) / 255.0
    label_train = buffer_csv.iloc[:, 0].values.astype(np.int)
    label_class_count = np.unique(label_train).shape[0]
    label_train = dense2one_hot(label_train, label_class_count).astype(np.float32)
    return data_train, label_train

## Define Network

In [11]:
tf.reset_default_graph()

X = tf.placeholder(tf.float32, [None, 784])
Y = tf.placeholder(tf.float32, [None, 10])

previous_output = None

with tf.variable_scope("fc1", reuse=tf.AUTO_REUSE):
    weight = tf.get_variable("weight", shape=[784, 1024], initializer=tf.truncated_normal_initializer(stddev=0.1), dtype=tf.float32)
    bias = tf.get_variable("bias", shape=[1024], initializer=tf.constant_initializer(value=0.1))
    previous_output = tf.nn.relu(tf.add(tf.matmul(X, weight), bias), name="output")

with tf.variable_scope("fc2", reuse=tf.AUTO_REUSE):
    weight = tf.get_variable("weight", shape=[1024, 10], initializer=tf.truncated_normal_initializer(stddev=0.1), dtype=tf.float32)
    bias = tf.get_variable("bias", shape=[10], initializer=tf.constant_initializer(value=0.1))
    previous_output = tf.nn.softmax(tf.add(tf.matmul(previous_output, weight), bias), name = "output")

tf.trainable_variables()

[<tf.Variable 'fc1/weight:0' shape=(784, 1024) dtype=float32_ref>,
 <tf.Variable 'fc1/bias:0' shape=(1024,) dtype=float32_ref>,
 <tf.Variable 'fc2/weight:0' shape=(1024, 10) dtype=float32_ref>,
 <tf.Variable 'fc2/bias:0' shape=(10,) dtype=float32_ref>]

## Training and Validation

In [12]:
TRAIN_SIZE = csv_train.shape[0]
BATCH_SIZE = 32
TRAIN_STEPS = np.floor(TRAIN_SIZE / BATCH_SIZE).astype(np.int)
EPOCH = 32
print("TRAIN_SIZE = {}, BATCH_SIZE = {}, TRAIN_STEPS = {}, EPOCH = {}".format(TRAIN_SIZE, BATCH_SIZE, TRAIN_STEPS, EPOCH))

TRAIN_SIZE = 42000, BATCH_SIZE = 32, TRAIN_STEPS = 1312, EPOCH = 32


In [13]:
# 损失函数
cross_entropy = tf.reduce_sum(tf.multiply(Y, tf.log(previous_output)))

# 优化函数
train_step = tf.train.AdamOptimizer(0.001).minimize(cross_entropy)

# 预测结果评估
correct_prediction = tf.equal(tf.argmax(Y, 1), tf.argmax(previous_output, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.Session() as session:
    session.run(tf.global_variables_initializer())

    for i in range(EPOCH):
        train_input, train_label = parse_train_csv(csv_train)
        for j in range(TRAIN_STEPS):
            lower = j * BATCH_SIZE
            upper = (j + 1) * BATCH_SIZE
            batch_input = train_input[lower : upper]
            batch_label = train_label[lower : upper]
            train_step.run(feed_dict={
                X: batch_input,
                Y: batch_label
            })

        print("epoch = {}, training accuracy = {}".format(i + 1, accuracy.eval(feed_dict={
            X: batch_input,
            Y: batch_label
        })))

epoch = 1, training accuracy = 0.15625
epoch = 2, training accuracy = 0.125
epoch = 3, training accuracy = 0.0625
epoch = 4, training accuracy = 0.0625
epoch = 5, training accuracy = 0.09375
epoch = 6, training accuracy = 0.15625
epoch = 7, training accuracy = 0.125
epoch = 8, training accuracy = 0.15625
epoch = 9, training accuracy = 0.0625
epoch = 10, training accuracy = 0.0625
epoch = 11, training accuracy = 0.21875
epoch = 12, training accuracy = 0.09375
epoch = 13, training accuracy = 0.03125
epoch = 14, training accuracy = 0.1875
epoch = 15, training accuracy = 0.09375
epoch = 16, training accuracy = 0.03125
epoch = 17, training accuracy = 0.09375
epoch = 18, training accuracy = 0.09375
epoch = 19, training accuracy = 0.03125
epoch = 20, training accuracy = 0.03125
epoch = 21, training accuracy = 0.09375
epoch = 22, training accuracy = 0.09375
epoch = 23, training accuracy = 0.0625
epoch = 24, training accuracy = 0.125
epoch = 25, training accuracy = 0.125
epoch = 26, training ac